# Technical Report
#### Collegiate Courses - New Textbook Library and Bookstore Data

### Overview
* We're taking raw data, extracting the information we want, transforming the information so it is in a universal format, and loading into a more user friendly database so analysts can more easily read and present the data. We set a goal of providing this data in ((2nd normal form)) for our analysts.

### Extract
#### We downloaded two files, one with textbook data, and another with college course data.
#### The first thing we did was use pandas to remove null values, and information that was not relevent to our requirements, such as "digital books", or isolating the "spring 2022" courses, etc.
* Selecting only certain columns to load: (or selecting null columns not to load). For example, if the source data has three columns (aka "attributes"), roll_no, age, and salary, then the selection may take only roll_no and salary. Or, the selection mechanism may ignore all those records where salary is not present (salary = null).

* Translating coded values: (e.g., if the source system codes male as "1" and female as "2", but the warehouse codes male as "M" and female as "F")

* Encoding free-form values: (e.g., mapping "Male" to "M")

* Deriving a new calculated value: (e.g., sale_amount = qty * unit_price)

* Sorting or ordering the data based on a list of columns to improve search performance

* Joining data from multiple sources (e.g., lookup, merge) and deduplicating the data

* Aggregating (for example, rollup — summarizing multiple rows of data — total sales for each store, and for each region, etc.)

* Generating surrogate-key values

* Transposing or pivoting (turning multiple columns into multiple rows or vice versa)

* Splitting a column into multiple columns (e.g., converting a comma-separated list, specified as a string in one column, into individual values in different columns)

* Disaggregating repeating columns

* Looking up and validating the relevant data from tables or referential files

* Applying any form of data validation; failed validation may result in a full rejection of the data, partial rejection, or no rejection at all, and thus none, some, or all of the data is handed over to the next step depending on the rule design and exception handling; many of the above transformations may result in exceptions, e.g., when a code translation parses an unknown code in the extracted data

### Transform
* We reformatted some numeric data to remove dollar signs, making them more user friendly as integers
* In pandas we also explored our data to make sure the formatting would be functional in all categories
* Selecting only certain columns to load: (or selecting null columns not to load). For example, if the source data has three columns (aka "attributes"), roll_no, age, and salary, then the selection may take only roll_no and salary. Or, the selection mechanism may ignore all those records where salary is not present (salary = null).

* Translating coded values: (e.g., if the source system codes male as "1" and female as "2", but the warehouse codes male as "M" and female as "F")

* Encoding free-form values: (e.g., mapping "Male" to "M")

* Deriving a new calculated value: (e.g., sale_amount = qty * unit_price)

* Sorting or ordering the data based on a list of columns to improve search performance

* Joining data from multiple sources (e.g., lookup, merge) and deduplicating the data

* Aggregating (for example, rollup — summarizing multiple rows of data — total sales for each store, and for each region, etc.)

* Generating surrogate-key values

* Transposing or pivoting (turning multiple columns into multiple rows or vice versa)

* Splitting a column into multiple columns (e.g., converting a comma-separated list, specified as a string in one column, into individual values in different columns)

* Disaggregating repeating columns

* Looking up and validating the relevant data from tables or referential files

* Applying any form of data validation; failed validation may result in a full rejection of the data, partial rejection, or no rejection at all, and thus none, some, or all of the data is handed over to the next step depending on the rule design and exception handling; many of the above transformations may result in exceptions, e.g., when a code translation parses an unknown code in the extracted data

### Load
* We used PostgreSQL to upload our data into a database called ---------- and tables called ---------&----------
* We then created file -----------.sql to create a junction table with a many to many relationship so that entities from either table could be used to uniquely call items from the other.